In [3]:
import sys, os, re
import numpy as np
import matplotlib.pyplot as plt

from skimage.color import label2rgb
from skimage.measure import label, regionprops
from skimage import morphology
from scipy import ndimage

from deeplate.MMdata import MMData

In [2]:
data_folder = '/scicore/home/nimwegen/urchuegu/projects/prj_noise_across_conditions_main/cell_size/data/20170126/201710126_lowerbound/'
folder = '/scicore/home/nimwegen/witzg/DeepPlateAnalysis/usingsubset6/Analysis_201710126_lowerbound/'

In [3]:
posid = [re.findall('(?<=maskcorrect_)(\d+)(?=\.npy)',f) for f in os.listdir(folder)]
posid = [int(x[0]) for x in posid if len(x)>0]

In [5]:
MMobj = MMData(folder = data_folder)

In [ ]:
print(MMobj.channels)

In [ ]:
#based on the above outpout set fluo and bright field channels
fluo_ch = 0
bf_ch = 1

In [ ]:
mid_plane = int((MMobj.num_planes[bf_ch]-1)/2)

In [13]:
mask = label(np.load(folder+'maskcorrect_'+str(posid[0])+'.npy'))
all_correlations = np.empty((mask.shape[0],mask.shape[1],len(posid)))
all_masks = np.empty((mask.shape[0],mask.shape[1],len(posid)))
all_weights = np.empty((mask.shape[0],mask.shape[1],len(posid)))

all_planes = np.empty((mask.shape[0],mask.shape[1],3, len(posid)))

for i in range(len(posid)):
    all_masks[:,:,i] = np.load(folder+'maskcorrect_'+str(posid[i])+'.npy')
    all_weights[:,:,i] = np.load(folder+'weightcorrect_'+str(posid[i])+'.npy')
    all_correlations[:,:,i] = np.load(folder+'corr_'+str(posid[i])+'.npy')
    
    mid = np.random.randint(mid_plane-5,mid_plane+6)
    for j in range(3):
        plane = mid-1+i
        all_planes[:,:,j,i] = MMobj.get_image(frame=0,channel=bf_ch,plane=plane,position=posid[i], compress = 1)
    

In [ ]:
numsplits = int(all_masks.shape[0]/128)
numimages = all_masks.shape[2]

#Definition of structures containing training and test datasets
#Dimensions and size of training and test datasets
channels = 3
image_rows = 128
image_cols = 128
num_test = 20
num_train = 2*((numsplits**2*numimages)-num_test)


imgs = np.ndarray((num_train, image_rows, image_cols,channels), dtype=np.float64)
imgs_mask = np.ndarray((num_train, image_rows, image_cols), dtype=np.uint8)
imgs_weight = np.ndarray((num_train, image_rows, image_cols), dtype=np.uint8)
imgs_test = np.ndarray((num_test, image_rows, image_cols,channels), dtype=np.float64)
imgs_id = np.ndarray((num_test, ), dtype=np.int32)

listpos = []
for k in range(numimages):
    for i in range(numsplits):
        for j in range(numsplits):
            listpos.append([i,j,k])
            
shuffle = np.random.choice(len(listpos), len(listpos), replace=False)

all_weights[all_weights==1] = 200
all_weights[all_masks==1] = 20
all_weights[all_weights==0] = 1

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
plt.imshow(all_planes[:,:,0,3],cmap= 'gray')
plt.imshow(all_weights[:,:,3],alpha = 0.5,cmap='Reds')
plt.show()

In [ ]:
counter_test = 0
for counter in range(len(listpos)):
    i=listpos[shuffle[counter]][0]
    j=listpos[shuffle[counter]][1]
    k=listpos[shuffle[counter]][2]
    
    img = all_planes[i*128:(i+1)*128,j*128:(j+1)*128,:,k]
    randint = np.random.rand() 
    if np.random.rand()<0.5: randint = 10*randint
    #randint = 100*randint
    img = img*randint
    img_mask = 255*all_masks[i*128:(i+1)*128,j*128:(j+1)*128,k]
    img_weight = all_weights[i*128:(i+1)*128,j*128:(j+1)*128,k]
            
    if 2*counter<num_train:
        if channels==1:
            imgs[2*counter] = img[..., np.newaxis]
            img = img.T
            imgs[2*counter+1] = img[..., np.newaxis]
        else:
            imgs[2*counter] = img
            img = img.T
            imgs[2*counter+1] = img
        imgs_mask[2*counter] = img_mask
        img_mask = img_mask.T
        imgs_mask[2*counter+1] = img_mask
        imgs_weight[2*counter] = img_weight
        img_weight = img_weight.T
        imgs_weight[2*counter+1] = img_weight
    else:
        if channels==1:
            imgs_test[counter_test] = img[..., np.newaxis]
        else:
            imgs_test[counter_test] = img

        imgs_id[counter_test] = counter_test
        counter_test+=1